In [39]:
import requests
import json
from datetime import datetime
import threading
from http.server import HTTPServer, BaseHTTPRequestHandler
import webbrowser
from urllib.parse import urlparse, parse_qs
import yaml

CLIENT_ID = None
CLIENT_SECRET = None

AUTH_CODE = None
ACCESS_TOKEN = None
REFRESH_TOKEN = None

class MyHandler(BaseHTTPRequestHandler):

    def do_GET(self):
        global AUTH_CODE
        
        query_components = parse_qs(urlparse(self.path).query)
        AUTH_CODE = query_components['code']
        self.send_response(200)
        self.end_headers()
        self.wfile.write(str.encode('Received auth code: {}'.format(AUTH_CODE)))
        
        # shutting down
        self.server.running = False

        
class MainServer:
    def __init__(self, port = 8123):
        self._server = HTTPServer(('0.0.0.0', port), MyHandler)
        self._thread = threading.Thread(target=self.run)
        self._thread.deamon = True 

    def run(self):
        self._server.running = True
        while self._server.running:
            self._server.handle_request()

    def start(self):
        self._thread.start()

    def shut_down(self):
        self._thread.close()
        

def reset_tokens():
    global AUTH_CODE, ACCESS_TOKEN, REFRESH_TOKEN
    AUTH_CODE = None
    ACCESS_TOKEN = None
    REFRESH_TOKEN = None


def setup(client_id=None, client_secret=None):
    global CLIENT_ID, CLIENT_SECRET
    
    if client_id is None and client_secret is None:
        with open('../credentials') as file:
            creds = yaml.full_load(file)
            CLIENT_ID = creds['client_id']
            CLIENT_SECRET = creds['client_secret']
    else:
        CLIENT_ID = client_id
        CLIENT_SECRET = client_secret
    
    reset_tokens()
    
    server = MainServer()
    server.start()
    webbrowser.open('https://www.strava.com/api/v3/oauth/authorize?response_type=code&client_id={}&redirect_uri=http://localhost:8123/give-me-token&scope=activity:read_all,profile:read_all&approval_prompt=force'.format(CLIENT_ID))
    

def refresh_token():
    global ACCESS_TOKEN, REFRESH_TOKEN
    url = 'https://www.strava.com/api/v3/oauth/token'
    
    payload = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': 'authorization_code' if REFRESH_TOKEN is None else 'refresh_token',
        'code': AUTH_CODE if REFRESH_TOKEN is None else REFRESH_TOKEN
    }
    
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    print(payload)

    response = requests.request("POST", url, headers=headers, data = payload)
    
    if response.status_code > 299:
        raise Exception(response.status_code, response.text)
    
    parsed_response = json.loads(response.text)
    ACCESS_TOKEN = parsed_response['access_token']
    REFRESH_TOKEN = parsed_response['refresh_token']
    
    print(parsed_response)
    
    return 'expires_at={}'.format(datetime.fromtimestamp(parsed_response['expires_at']))



def api_request(endpoint, retries=3):
    url = "https://www.strava.com/api/v3" + endpoint

    payload = {}
    headers = {
      'Authorization': 'Bearer {}'.format(ACCESS_TOKEN)
    }

    response = requests.request("GET", url, headers=headers, data = payload)
    
    if response.status_code > 299:
        raise Exception(response.status_code, response.text)
    
    return json.loads(response.text)


In [43]:
"""
Setup authorization from credentials file
"""
setup()

127.0.0.1 - - [23/Feb/2020 22:25:40] "GET /give-me-token?state=&code=c907e976e1914a03d8becc74a76346413bb53aa6&scope=read,activity:read_all,profile:read_all HTTP/1.1" 200 -


In [44]:
"""
Refresh access token
"""
refresh_token()

{'client_id': 20539, 'client_secret': '954df898cdd4d90a239040d2124aa2a6afceab41', 'grant_type': 'authorization_code', 'code': ['c907e976e1914a03d8becc74a76346413bb53aa6']}
{'token_type': 'Bearer', 'expires_at': 1582509930, 'expires_in': 16782, 'refresh_token': '5f2e8b899c82e4b28b93a0fb9c34aee6c2a78074', 'access_token': 'e239ffd0d8bf523d9feab34bf1380176352fc5a9', 'athlete': {'id': 2706822, 'username': 'mat_kuz', 'resource_state': 2, 'firstname': 'Mateusz', 'lastname': 'Kuźmik', 'city': 'Rzeszów', 'state': 'Województwo podkarpackie', 'country': 'Polska', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2013-08-01T12:38:14Z', 'updated_at': '2020-02-07T13:00:06Z', 'badge_type_id': 1, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/2706822/13107580/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/2706822/13107580/1/large.jpg', 'friend': None, 'follower': None}}


'expires_at=2020-02-24 03:05:30'